In [294]:
import pandas as pd
from IPython.display import Markdown
import re

In [295]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [296]:
# Reload dotenv in notebook
from dotenv import load_dotenv
dot_env_loaded = load_dotenv()
dot_env_loaded

True

# 1. Lyrics Database

In [240]:
lyrics_df_all = pd.read_csv('../raw_data/20250609_17k_lyrics_eng_fr.csv')

In [241]:
lyrics_df_all.drop(columns='Unnamed: 0', inplace=True)
lyrics_df_all.reset_index(drop=True, inplace=True)

In [242]:
lyrics_df_all.isna().sum()

artist                     0
track_title_clean          0
lyrics_clean               3
artist_lower               0
track_title_clean_lower    0
dtype: int64

In [243]:
lyrics_df_all.groupby('artist').count()['lyrics_clean'].sort_values(ascending=False).head(50)

artist
Booba                       90
Marilyn Manson              89
Tim McGraw                  89
Rascal Flatts               89
Kenny Chesney               86
Grateful Dead               85
Shy'm                       85
George Strait               83
Phish                       83
Muse                        80
Christine and the queens    78
Kery James                  78
Def Leppard                 73
Lionel Richie               73
Jenifer                     72
Carla Bruni                 72
Lara Fabian                 71
Rush                        70
Gradur                      69
Lefa                        68
Zazie                       68
Aya Nakamura                68
Weezer                      68
PNL                         68
Slayer                      67
Korn                        67
Norah Jones                 67
Foo Fighters                66
Céline Dion                 66
Hamza                       66
Lynyrd Skynyrd              66
Megadeth                    65
M

## 1.1 Clean lyrics

In [230]:
lyrics_df_all

,artist,track_title_clean,lyrics_clean,artist_lower,track_title_clean_lower
0,ABBA,"Andante, Andante","Take it easy with me, please Touch me gently l...",abba,"andante, andante"
1,ABBA,As Good As New,I'll never know why I had to go Why I had to p...,abba,as good as new
2,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...,abba,bang-a-boomerang
3,ABBA,Cassandra,Down in the street they're all singing and sho...,abba,cassandra
4,ABBA,Chiquitita,"Chiquitita, tell me what's wrong You're enchai...",abba,chiquitita
...,...,...,...,...,...
17299,Therapie Taxi,Candide Crush,"Je suis si sage-sage d'habitude, mon chéri Com...",therapie taxi,candide crush
17300,Therapie Taxi,J'en ai marre,Et j'en ai marre de toi Je sais c'est dur mais...,therapie taxi,j'en ai marre
17301,Therapie Taxi,Avec ta zouz,"N'aie pas le seum, fais-moi la bise Ce soir c'...",therapie taxi,avec ta zouz
17302,Minuit,Flash,"Lumière crue, lumière sans, flash! Étrange sen...",minuit,flash


In [251]:
lyrics_df_all.dropna(inplace=True)

In [252]:
import re

def no_backslash(lyrics):
    lyrics = lyrics.replace('\\', '')
    return lyrics


In [253]:
lyrics_df_all['lyrics_clean'] = lyrics_df_all['lyrics_clean'].apply(no_backslash)

In [273]:
lyrics_df_all[(lyrics_df_all.artist == 'Booba') & (lyrics_df_all.track_title_clean == 'Bénigni')]['lyrics_clean']

16194    Avant, j'voulais la plus sexy, maintenant j've...
Name: lyrics_clean, dtype: object

# 3. Download Gemini model

In [297]:
import os
google_api_key_exists = 'GOOGLE_API_KEY' in os.environ
google_api_key_exists

True

## 3.1 Import GenAI Gemini model

In [299]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

## 3.2 Create a tool to get lyrics

In [ ]:
from langchain_core.tools import tool

@tool
def get_lyrics(artist_name : str) -> str:
    """ Get song titles and lyrics of a specific artist's name.
    Use the artist name in the query as artist_name """
    songs = lyrics_df_all[lyrics_df_all['artist'].isin([artist_name])]
    if songs.empty:
        return "No songs found for this artist."

    results = []
    for _, row in songs.iterrows():
        results.append(f"Title: {row['track_title_clean']}\nLyrics: {row['lyrics_clean']}\n")
    return "\n".join(results)

In [341]:
tools = [get_lyrics]

## 3.3 Create a system prompt

In [ ]:
system_prompt = """
    With the name of an artist as an input, and the lyrics of their songs from the get_lyrics tool,
    you are tasked to summarize the main themes in their lyrics. The output would follow the following format:
    Here are the 5 main themes in Adele's song “Chasing Pavements” with quotes to illustrate each:
    1. Uncertainty and Doubt
        The song revolves around the question of whether to keep pursuing a relationship or give up. This doubt is clear in the repeated lines:
        “Should I give up? Or should I just keep chasing pavements? Even if it leads nowhere.”

    2. Love vs. Rationality
        Adele expresses the tension between emotional conviction and logical thinking. She knows it’s love but questions if it’s worth continuing:
        “This ain't lust, I know this is love, but if I tell the world I'll never say enough.”

    3. Fear of Waste and Futility
        There is a strong fear that continuing the pursuit might be pointless or a waste of effort:
        “Or would it be a waste? Even if I knew my place, should I leave it there?”

    4. Hope and Perseverance
        Despite doubts, she shows a desire to keep trying, to keep “chasing pavements,” hoping for a positive outcome even if the path is unclear:
        “I build myself up and fly around in circles, waiting as my heart drops.”

    5. Self-reflection and Decision-Making
        The song is about introspection and the difficult process of making a decision about love and whether to keep going or move on:
        “I've made up my mind, don't need to think it over, if I'm wrong, I am right.”

    These themes together create a nuanced emotional landscape about love, uncertainty, and the struggle between hope and letting go.
    """

## 3.4 Create an agent executor

### 3.4.1 Test Jule's version

In [354]:
# Import
from langgraph.prebuilt import create_react_agent

# Create the `agent_executor` with a model and tools
agent_executor = create_react_agent(model, tools)

In [355]:
from langchain_core.messages import HumanMessage

query = "Summarize booba's songs"
response = agent_executor.invoke(
    {"messages": [HumanMessage(content=query)]}
)
response["messages"][-1].content

'Sorry, I cannot fulfill the request. The available tools do not have the ability to summarize songs. I can only get song titles and lyrics of a specific artist.'

### 3.4.2 Test upgraded version

In [356]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType
from langchain.schema import HumanMessage

In [373]:
# Initialize the agent with your tool and model
agent_executor = initialize_agent(
    tools,
    model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

artist = 'Adele'

query = f"Summarize the top 5 main themes in {artist}'s lyrics. For each theme please quote some lyrics"
response = agent_executor.run(query)
Markdown(response)



> Entering new AgentExecutor chain...
I need to get Adele's lyrics to identify and summarize the main themes. Then I will provide lyrical quotes to support each theme.
Action: get_lyrics
Action Input: Adele
Observation: Title: All I Ask
Lyrics: [Verse 1] I will leave my heart at the door I won't say a word They've all been said before, you know So why don't we just play pretend Like we're not scared of what's coming next Or scared of having nothing left [Pre-Chorus] Look, don't get me wrong I know there is no tomorrow All I ask is [Chorus] If this is my last night with you Hold me like I'm more than just a friend Give me a memory I can use Take me by the hand while we do what lovers do It matters how this ends Cause what if I never love again? [Verse 2] I don't need your honesty It's already in your eyes and I'm sure my eyes, they speak for me No one knows me like you do And since you're the only one that matters, tell me who do I run to? [Pre-Chorus] Now, don't get me wrong I know t

The top 5 main themes in Adele's lyrics are: Heartbreak and Loss ("Hello from the other side / I must've called a thousand times / To tell you I'm sorry, for everything that I've done"), Longing and Nostalgia ("Let me photograph you in this light / In case it is the last time that we might / Be exactly like we were before we realised / We were sad of getting old it made us restless"), Self-Reflection and Regret ("I know I have a fickle heart and a bitterness / And a wandering eye, and a heaviness in my head"), Hope and Resilience ("Never mind, I'll find someone like you / I wish nothing but the best for you, too"), and Vulnerability and Honesty ("Take your eyes off me so I can leave / I'm far too ashamed to do it with you watching me").

## 3.5 Prompt AI agent executor

In [ ]:
system_prompt = """
With the name of an artist as an input, and the lyrics of their songs from the get_lyrics tool,
you are tasked to summarize the top 5 themes in their lyrics.

Below is an example of the format to use for the response:
Here are the 5 main themes in {artist}'s songs:
1. Uncertainty and Doubt
    The song revolves around the question of whether to keep pursuing a relationship or give up. This doubt is clear in the repeated lines:
    “Should I give up? Or should I just keep chasing pavements? Even if it leads nowhere.”

2. Love vs. Rationality
    Adele expresses the tension between emotional conviction and logical thinking. She knows it’s love but questions if it’s worth continuing:
    “This ain't lust, I know this is love, but if I tell the world I'll never say enough.”

3. Fear of Waste and Futility
    There is a strong fear that continuing the pursuit might be pointless or a waste of effort:
    “Or would it be a waste? Even if I knew my place, should I leave it there?”

4. Hope and Perseverance
    Despite doubts, she shows a desire to keep trying, to keep “chasing pavements,” hoping for a positive outcome even if the path is unclear:
    “I build myself up and fly around in circles, waiting as my heart drops.”

5. Self-reflection and Decision-Making
    The song is about introspection and the difficult process of making a decision about love and whether to keep going or move on:
    “I've made up my mind, don't need to think it over, if I'm wrong, I am right.”

These themes together create a nuanced emotional landscape about love, uncertainty, and the struggle between hope and letting go.
"""

In [378]:
agent_2 = create_react_agent(model, tools, prompt=system_prompt)

In [ ]:
artist = 'Dadju'

query = f"Synthétise les top 5 thèmes des paroles de {artist} et score leur pertinence. Pour chaque thème cite 3 différentes paroles"

for step in agent_2.stream(
    {"messages": [HumanMessage(content=query)]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Synthétise les top 5 thèmes des paroles de Dadju et score leur pertinence. Pour chaque thème cite 3 différentes paroles
================================== Ai Message ==================================

Bien sûr, je peux faire ça.
Tool Calls:
  get_lyrics (4291bf6a-9ca3-4c71-91d4-5c0dcd82d866)
 Call ID: 4291bf6a-9ca3-4c71-91d4-5c0dcd82d866
  Args:
    artist_name: Dadju
================================= Tool Message =================================
Name: get_lyrics

Title: J'ai zayé
Lyrics: J'sais qu'elle m'apprécie comme pas possible Plus j'insiste, plus elle est indécise Mais je dois continuer, continuer Devant la petite, oh, j'ai zayé, oh C'est la fierté qui va nous die, yeah, oh Taykee de Taykee Die, yeah, oh Approche-toi de moi, viens kulosa, j'connais pas la chorée, apprends-moi pretty dance, ah J'prendrais soin de toi un petit peu mieux qu'l'a fait ton père Viens, on décale là où y aura beaucoup de

In [395]:
Markdown(step["messages"][-1].pretty_print())

================================== Ai Message ==================================

Voici les 5 principaux thèmes des chansons de Dadju :

1.  **Relations Amoureuses Tumultueuses et Infidélité :**
    *   Les chansons explorent souvent la complexité des relations amoureuses, avec des thèmes d'infidélité, de jalousie et de ruptures.
    *   "J'ai fais le con, je t'ai menti, menti, tu l'as pas vu, pas senti, senti Donc j’ai continuer de te mentir, mentir, mais c’était de pire en pire" (Ma Faute)
    *   "On s\'fait la guerre sans même encore et encore sans même vouloir la gagner C\'est chacun son tour qui est ce qui se prendra la première balle du barillet" (Amour Toxic)
    *   "J'te dis que j'ai fait du sale à je n'sais combien de femmes Et j'ai peur que tu deviennes mon retour de flamme" (Jaloux Remix)

2.  **Difficulté à Exprimer ses Sentiments et Fierté :**
    *   Le thème de la difficulté à exprimer ses émotions et de la fierté qui empêche de montrer ses vrais sentiments revient sou

<IPython.core.display.Markdown object>